# 1. Lattice attack for Matrix NTRU to recover the private key (usually a permutation of it)


### 1.1. Outline of the attack in the lattice

1. Generate a key pair for matrix NTRU with parameters n e q (p fixed at 3)

2. Use the public key H to construct the lattice L.

3. Apply BKZ to L and obtain Lreduced

4. Check how many lines of the private key X are contained into (check + line and - line) and return it. 


### 1.2 Parameter sets for attack

- ntruencrypt round3 - n 509 q 2048, n 677 q 2048, n 821 q 4096. most secure one is the last one, so this gives us a guideline for testing the matrix ntru system.
- choose, p = 3, q = 4096, n in {5,10,20,30,40,50,60,...,100}
- 1000 repetitions for each scenario.
- save into an array with dimensions len(n_values), repetitions
- (done in R after exporting results as csv). At the end make a histogram where for each value of n we have categories of sucess divided as follows: 0% of lines recovered, (0,25%), [25%,50%), [50%,75%), [75%,100%), 100% of lines recovered,

Run again with other values of q. E.g. 

# 2. Start of CODE

### 2.1 Import libraries and define functions to:

- create the associated lattice
- use the lattice and run BKZ to find a candidate for the private key
- find out the attack sucess rate

In [ ]:
load("matrix_ntru_system.sage")

In [ ]:
import numpy as np
def matrixNTRULattice(H,n,p,q):
    # returns matrix NTRU corresponding lattice from public key
    AB = np.concatenate((identity_matrix(n), p.inverse_mod(q) * H), axis=1)
    CD = np.concatenate((0*identity_matrix(n), q*identity_matrix(n)), axis=1)
    L = matrix(ZZ,np.concatenate((AB,CD),axis=0))
    return L

def count_corresponding_lines(A, B):
    """
    This function counts the number of lines in B that correspond 
    to lines in A.

    Args:
      A: A 2D numpy array representing matrix A.
      B: A 2D numpy array representing matrix B.
      comparison_method: A function that takes two lines (rows) 
      as input and returns True if they correspond and 
      False otherwise.

    Returns:
      An integer representing the number of corresponding lines in B.
    """
    count = 0
    for lineA in A:
        for lineB in B:
            #print(lineA,lineB,lineA == lineB or lineA == -lineB)
            count += int(lineA == lineB or lineA == -lineB)
    return count


def matrixNTRULatticeAttack(H,n,p,q,X,Y):
    """
    creates lattice and reduces it. Afterwards, returns how 
    many lines of private key pair (X|Y) is contained in the 
    reduced lattice

    Returns:
    [a,b] where
    a: lines of private key pair (X|Y) is contained in the reduced lattice
    b: 0 if bkz terminated sucessfully in executing, and 1 if it failed
    """
    L = matrixNTRULattice(H,n,p,q)
    try:
        LBKZ = L.BKZ()
        XY = matrix(ZZ,np.concatenate((X,Y),axis=1))
        return([count_corresponding_lines(XY,LBKZ),0])
    except:
        # first line says we found zero lines and second means, bkz failed
        return([0,1])

    

### 2.2 Simulation study of attack performance for fixed q = 4096 and varying n

In [ ]:
nvalues = [5,10,20,30,40,50,60,70,80,90,100,110] # n = 115 and on bkz fails and we cannot run the attack. Consider going beyond 113. 
q = 4096 # q > 2n(p+1), even for n = 100, we can use, q > 600, e.g, 701 which is coprime with p = 3. 
rep = 100
result = matrix(ZZ,len(nvalues),rep)
resultBKZfail = matrix(ZZ,len(nvalues),rep)


import time, math
start_time = time.time()

for i in range(len(nvalues)):
    print('running for n = ' + str(nvalues[i]))
    setParameters([nvalues[i],q]) # parameters n and q.
    for j in range(rep):
        X,Y,Xp,Xq,H = keygen()
        result[i,j],resultBKZfail[i,j] = matrixNTRULatticeAttack(H,n,p,q,X,Y)
        #print(nvalues[i],result[i,j])
        
print("--- %s seconds ---" % (time.time() - start_time))        

### 2.3 save result in a csv file

In [ ]:
# Define the file path where you want to save the CSV file
import csv
path_result = "/Users/thiago/work/latticesgo/matrix ntru experimental study/result_q_32.csv"  # Update this with your desired file path
with open(path_result, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(result)


path_result_BKZfail = "/Users/thiago/work/latticesgo/matrix ntru experimental study/result_q_32_bkzfail.csv"  # Update this with your desired file path
with open(path_result_BKZfail, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(resultBKZfail)

### 2.4 Simulation study for several values of n and q

In [ ]:
nvalues = [5,10,20,30,40] #[5,10,20,30,40,50,60,70,80,90,100,110] # n = 115 and on bkz fails and we cannot run the attack. Consider going beyond 113. 
qvalues =  [32,64,79,128] # [32,64,79,128,256,307,512,701,1024,2048] # q > 2n(p+1), even for n = 100, we can use, q > 600, e.g, 701 which is coprime with p = 3. 
rep = 10
result = matrix(ZZ,len(nvalues),rep)
resultBKZfail = matrix(ZZ,len(nvalues),rep)
resultList = [[0,result,resultBKZfail]]*len(qvalues) # qvalue, results, resultBKZfail

for k in range(len(qvalues)):
    print("====================")
    result = matrix(ZZ,len(nvalues),rep)
    resultBKZfail = matrix(ZZ,len(nvalues),rep)
    for i in range(len(nvalues)):
        setParameters([nvalues[i],qvalues[k]]) # parameters n and q.
        getParameters()
        for j in range(rep):
            X,Y,Xp,Xq,H = keygen()
            result[i,j],resultBKZfail[i,j] = matrixNTRULatticeAttack(H,n,p,q,X,Y)
    resultList[k] = [qvalues[k],result,resultBKZfail]